# Captain's Log

In [2]:
# Setup

# pip3 install urllib3 # Might not have worked. Hard to say
# pip install urllib3
# pip3 install requests



In [51]:
#
#   MODEL - data structures
#

# Score data structure
class Score:
    def __init__(self, total, item):
        self.total = total
        self.item = item
    def __repr__(self):
        return repr((self.total, self.item))

# SuperPoint data structure
class SuperPoint:
    def __init__(self, item):
        self.items = []
        self.items.append(item)
    def __repr__(self):
        return repr((self.items))



In [53]:
#
# UTILITY methods
#
from math import sqrt

# Get the distance between two points
def getDistance(loc1, loc2):
    dist = sqrt( (loc2['lon'] - loc1['lon'])**2 + (loc2['lat'] - loc1['lat'])**2 )
    return dist

# Are two points close enough
def closeEnough(loc1, loc2, limitDistance):
    thisDistance = getDistance(loc1, loc2)
    return thisDistance <= limitDistance

# Count the number of points within a distance of a point
def getScore(point, allPoints, limitDistance):
    result = 0
    for other in allPoints:
        if closeEnough(point, other, limitDistance):
            result += 1
    return result

# Convert anything to json
import json
def getJson(something):
    json_string = json.dumps(something)
    return json_string



# A super point is based off of an initial point
def createSuperPoint():
    point = sorted_scores[0].item
    sorted_scores.pop(0)
    sp = SuperPoint(point)
    i = 0
    while i < len(sorted_scores):
        other = sorted_scores[i].item
        if closeEnough(point, other, arbitaryDistanceLimit):
            sp.items.append(other)
            sorted_scores.pop(0)
            i -= 1
        i += 1
    return sp


# All subsequent points must also be checked for near-neighbors
def nourishSuperPoint(sp):
    for point in sp.items:
        i = 0
        popped = 0
        iniTot = len(sorted_scores)
        while i < (iniTot - popped):
            other = sorted_scores[i].item
            print(getJson(other['id']))
            if closeEnough(point, other, arbitaryDistanceLimit):
                sp.items.append(other)
                print("Adding:", other['id'])
                print("len before: ", len(sorted_scores))
                sorted_scores.pop(0)
                print("len after: ", len(sorted_scores))
                popped += 1
                i -= 1
            i += 1




In [54]:
#
# SETUP
#

import json, urllib.request, requests
from collections import namedtuple

def getLocations(url):
    r = requests.get(url)
    j = json.loads(r.text)
    return j

sample_locations = getLocations('https://geo-pg.captainjustin.space/transactions_r_sample')

# Decide on accuracy
arbitaryDistanceLimit = 0.001


In [55]:
### Let's score each point by how many other points are within `arbitaryDistanceLimit`


# We'll keep scores here in an array here
scores = [] 


# Get the score for each location
for l in sample_locations:
    total = getScore(l, sample_locations, arbitaryDistanceLimit)
    scores.append(Score(total, l))

sorted_scores = sorted(scores, key=lambda score: score.total, reverse=True)

print("Scoring is done and scores are sorted")
print(len(sorted_scores))


Scoring is done and scores are sorted
1036


In [56]:
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
import plotly.io as pio

import os
import numpy as np

init_notebook_mode(connected=True)


N = 0
x = np.random.rand(N)
y = np.random.rand(N)
colors = np.random.rand(N)
sz = np.random.rand(N)*30

for i in sorted_scores:
    x = np.append(x, i.item['lon'])
    y = np.append(y, i.item['lat'])
    sz = np.append(sz, 10 + (i.total / 10))
    colors = np.append(colors, i.total)


fig = go.Figure()
fig.add_scatter(x=x,
                y=y,
                mode='markers',
                marker={'size': sz,
                        'color': colors,
                        'opacity': 0.2,
                        'colorscale': 'Viridis'
                       });
iplot(fig)


In [58]:
print("..About to start: ", len(sorted_scores))
sp1 = createSuperPoint()
print("..After CREATE superPoint has: ", len(sp1.items))
print("..After CREATE sorted_scores has: ", len(sorted_scores))
# nourishSuperPoint(sp1)
print("..After NOURISH superPoint has: ", len(sp1.items))
print("..After NOURISH sorted_scores has: ", len(sorted_scores))
print("..Here is what is in the SP: ", len(sorted_scores))
print(getJson(sp1.items))



..About to start:  646
..After CREATE superPoint has:  364
..After CREATE sorted_scores has:  282
..After NOURISH superPoint has:  364
..After NOURISH sorted_scores has:  282
..Here is what is in the SP:  282
[{"id": "1550", "lon": 27.951866799999998, "lat": -26.1190605, "tim": "2018-09-11T11:00:23+00:00", "amount": 0.0}, {"id": "1559", "lon": 27.951835199999998, "lat": -26.1190571, "tim": "2018-09-11T10:40:58+00:00", "amount": 0.0}, {"id": "621", "lon": 27.9510783, "lat": -26.1191086, "tim": "2018-09-13T08:19:25+00:00", "amount": 0.0}, {"id": "4275", "lon": 27.9510798, "lat": -26.1191149, "tim": "2018-09-06T06:34:32+00:00", "amount": 0.0}, {"id": "4905", "lon": 27.951207399999998, "lat": -26.118966099999998, "tim": "2018-09-05T06:27:56+00:00", "amount": 0.0}, {"id": "8151", "lon": 27.9514052, "lat": -26.1190362, "tim": "2018-08-30T11:08:25+00:00", "amount": 0.0}, {"id": "8828", "lon": 27.951202799999997, "lat": -26.1193068, "tim": "2018-08-29T09:05:52+00:00", "amount": 0.0}, {"id": "2

In [59]:
leftOver = []
for i in sorted_scores:
    leftOver.append(i.item)
print(getJson(leftOver))

[{"id": "1482", "lon": 27.9509144, "lat": -26.119628199999998, "tim": "2018-09-11T13:58:49+00:00", "amount": 0.0}, {"id": "4343", "lon": 27.950837099999998, "lat": -26.1196526, "tim": "2018-09-06T05:32:54+00:00", "amount": 0.0}, {"id": "1098", "lon": 27.9507725, "lat": -26.1196484, "tim": "2018-09-12T09:36:22+00:00", "amount": 0.0}, {"id": "5975", "lon": 27.9508095, "lat": -26.1196555, "tim": "2018-09-03T08:26:37+00:00", "amount": 0.0}, {"id": "8371", "lon": 27.950739, "lat": -26.1195646, "tim": "2018-08-30T05:37:34+00:00", "amount": 0.0}, {"id": "9407", "lon": 27.9507473, "lat": -26.1195995, "tim": "2018-08-28T09:31:38+00:00", "amount": 0.0}, {"id": "661", "lon": 27.951077599999998, "lat": -26.1197366, "tim": "2018-09-13T07:08:06+00:00", "amount": 0.0}, {"id": "1721", "lon": 27.951077599999998, "lat": -26.1197366, "tim": "2018-09-11T05:58:31+00:00", "amount": 0.0}, {"id": "3581", "lon": 27.951077599999998, "lat": -26.1197366, "tim": "2018-09-07T08:10:26+00:00", "amount": 0.0}, {"id": 